In [ ]:
!git clone https://github.com/piEsposito/blitz-bayesian-deep-learning.git

In [ ]:
!(cd blitz-bayesian-deep-learning/ && pip install .)

In [ ]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator

from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
class SudokuDataset(Dataset):
    """Sudoku dataset."""

    def __init__(self, csv_file):
        """
        Args:
            csv_file (string): Path to the csv file with puzzles.
        """
        self.sudoku_frame = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.sudoku_frame)

    def __getitem__(self, idx):
        
        
        x = one_hot_encode(self.sudoku_frame.loc[idx].puzzle)
        y = one_hot_encode(self.sudoku_frame.loc[idx].solution)
        
        sample = {'x': x, 'y': y}

        return sample
    
def one_hot_encode(s):
    zeros = torch.zeros((81, 9), dtype=torch.float)
    for a in range(81):
        zeros[a, int(s[a]) - 1] = 1 if int(s[a]) > 0 else 0
    return zeros

In [ ]:
del dataset
dataset = SudokuDataset("/kaggle/input/sudoku/sudoku.csv")

In [ ]:
trainloader = DataLoader(dataset, batch_size=256,
                                        shuffle=True, num_workers=2)

In [ ]:
def create_constraint_mask():
    constraint_mask = torch.zeros((81, 3, 81), dtype=torch.float)
    # row constraints
    for a in range(81):
        r = 9 * (a // 9)
        for b in range(9):
            constraint_mask[a, 0, r + b] = 1

    # column constraints
    for a in range(81):
        c = a % 9
        for b in range(9):
            constraint_mask[a, 1, c + 9 * b] = 1

    # box constraints
    for a in range(81):
        r = a // 9
        c = a % 9
        br = 3 * 9 * (r // 3)
        bc = 3 * (c // 3)
        for b in range(9):
            r = b % 3
            c = 9 * (b // 3)
            constraint_mask[a, 2, br + bc + r + c] = 1

    return constraint_mask

In [ ]:
#@variational_estimator - uncomment if bayesian
class SudokuSolver(nn.Module):
    def __init__(self, constraint_mask, n=9, hidden1=128, bayesian=False):
        super(SudokuSolver, self).__init__()
        self.constraint_mask = constraint_mask.unsqueeze(-1).unsqueeze(0)
        self.n = n
        self.hidden1 = hidden1

        # Feature vector is the 3 constraints
        self.input_size = 3 * n
                self.a1 = nn.ReLU()
        if bayesian:
            self.l1 = BayesianLinear(self.input_size,
                            self.hidden1, bias=False)
            self.l2 = BayesianLinear(self.hidden1,
                            n, bias=False)
        else:
            self.l1 = nn.Linear(self.input_size,
                                self.hidden1, bias=False)
            self.l2 = nn.Linear(self.hidden1,
                                n, bias=False)
        self.softmax = nn.Softmax(dim=1)

    # x is a (batch, n^2, n) tensor
    def forward(self, x, return_orig=False):
        n = self.n
        bts = x.shape[0]
        c = self.constraint_mask
        min_empty = (x.sum(dim=2) == 0).sum(dim=1).max()
        x_pred = x.clone()
        for a in range(min_empty):
            # score empty numbers
            #print(x.view(bts, 1, 1, n * n, n).size(), x.unsqueeze(1).unsqueeze(1).size())
            constraints = (x.unsqueeze(1).unsqueeze(1) * c).sum(dim=3)
            # empty cells
            empty_mask = (x.sum(dim=2) == 0)

            f = constraints.reshape(bts, n * n, 3 * n)
            y_ = self.l1(f[empty_mask])
            y_ = self.l2(self.a1(y_))

            s_ = self.softmax(y_)

            # Score the rows
            x_pred[empty_mask] = s_

            s = torch.zeros_like(x_pred)
            s[empty_mask] = s_
            # find most probable guess
            score, score_pos = s.max(dim=2)
            mmax = score.max(dim=1)[1]
            # fill it in
            nz = empty_mask.sum(dim=1).nonzero().view(-1)
            mmax_ = mmax[nz]
            ones = torch.ones(nz.shape[0]).cuda()
            x.index_put_((nz, mmax_, score_pos[nz, mmax_]), ones)
        if return_orig:
            return x
        else:
            return x_pred

In [ ]:
constraint_mask = create_constraint_mask().cuda()
criterion = nn.MSELoss()
sudoku_solver = SudokuSolver(constraint_mask).cuda()

optimizer = optim.Adam(sudoku_solver.parameters(), lr=0.001)

In [ ]:
def evaluate_regression(regressor,X,y,):
    preds = regressor(X)
    errors = preds.max(dim=2)[1]\
                != y.max(dim=2)[1]
    return errors

In [ ]:
from tqdm import tqdm

In [ ]:
epochs = 100
loss_train = []
loss_val = []
eval_border = int(len(trainloader) * 0.9)
iterations = 0
for e in range(epochs):
    sudoku_solver.train()
    for i_batch, sudokus in enumerate(trainloader):
        x = sudokus['x'].cuda()
        y = sudokus['y'].cuda()
        if i_batch < eval_border:
            optimizer.zero_grad()
            output = sudoku_solver(x)
            loss = criterion(output, y)
#             loss = sudoku_solver.sample_elbo(inputs=x,
#                                    labels=y,
#                                    criterion=criterion,
#                                    sample_nbr=100)
            loss.backward()
            optimizer.step()
            loss_train.append(loss.item())
            if iterations % 100 == 0:
                print(f"epoch #{e} {iterations} iterations train - {loss.item()}")
            iterations+=1
        else:
            sudoku_solver.eval()
            loss_val.append(evaluate_regression(sudoku_solver, x, y).sum().item())
    print(f"epoch #{e} {iterations} iterations val - {loss_val[-1]}")